## Analyzing the Chinook Mock Database to Answer Sales Questions

Load sql extension and connect to the Chinook database

In [79]:
%load_ext sql
%sql sqlite:///chinook.db

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


'Connected: @chinook.db'

## Overview of the Chinook database

In [80]:
%%sql
SELECT 
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

 * sqlite:///chinook.db
Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


## Which Album Genres Sell the Most?

In [81]:
%%sql

WITH tracks_sold_usa AS
    (SELECT il.track_id track_id, SUM(il.quantity) quantity
       FROM invoice_line il
      INNER JOIN invoice i on i.invoice_id = il.invoice_id
      WHERE i.billing_country = "USA"
      GROUP BY track_id
     ),
    
     genre_sold AS
    (SELECT g.name name, 
            SUM(ts.quantity) quantity
       FROM genre g
       LEFT JOIN track t on t.genre_id=g.genre_id
      INNER JOIN tracks_sold_usa ts on ts.track_id=t.track_id
      GROUP BY g.name
      ORDER BY quantity DESC
    )
        

SELECT *, 
       ROUND((CAST(quantity AS FLOAT)/(SELECT SUM(quantity)FROM genre_sold))*100,2)||"%" percentage
FROM genre_sold;

 * sqlite:///chinook.db
Done.


name,quantity,percentage
Rock,561,53.38%
Alternative & Punk,130,12.37%
Metal,124,11.8%
R&B/Soul,53,5.04%
Blues,36,3.43%
Alternative,35,3.33%
Pop,22,2.09%
Latin,22,2.09%
Hip Hop/Rap,20,1.9%
Jazz,14,1.33%


Most sold genres in the USA are 1) Rock, 2) Metal, and 3) Alternative & Punk. 

## Analyzing Employee Sales Data: Do Variances in Sales Point to Employee Performance?

In [82]:
%%sql

WITH total_per_customer AS
     (SELECT customer_id, SUM(total) total
        FROM invoice
       GROUP BY customer_id
     ),
    
     sales_per_rep AS
    (SELECT c.support_rep_id rep_id, SUM(tpc.total) sales
       FROM customer c
       LEFT JOIN total_per_customer tpc ON tpc.customer_id=c.customer_id
      GROUP BY rep_id
     ),
    
    sales_per_employee AS
    (SELECT e.first_name||" "||e.last_name name, e.hire_date, "$"||ROUND(SUM(spr.sales),2) sales
       FROM employee e
       LEFT JOIN sales_per_rep spr on spr.rep_id=e.employee_id
      WHERE e.title = "Sales Support Agent"
      GROUP BY e.employee_id
      ORDER BY sales DESC
    )
    
SELECT *
  FROM sales_per_employee;
     

 * sqlite:///chinook.db
Done.


name,hire_date,sales
Jane Peacock,2017-04-01 00:00:00,$1731.51
Margaret Park,2017-05-03 00:00:00,$1584.0
Steve Johnson,2017-10-17 00:00:00,$1393.92


Variance in sales dollars per Sales Support Agent seems to correlate to the date of hire: reps hired earlier in time have more sales. Variance does not seem to be indicative of employee performance e.g. the 20% difference in sales between the top selling employee and the bottom selling employee roughly corresponds to the differences in their hiring dates.

## Analyzing Sales by Country: Which Countries Could be Targets for New Opportunity?

In [83]:
%%sql

WITH customer_sales AS 
    (SELECT customer_id, 
            COUNT(invoice_id) sales,
            SUM(total) sales_revenue 
       FROM invoice 
      GROUP BY customer_id
    ),
    
    country_sales AS 
    (SELECT country, 
            COUNT(cs.customer_id) customers, 
            SUM(sales) sales, 
            ROUND(SUM(sales_revenue),2) sales_amount
       FROM customer c
       LEFT JOIN customer_sales cs ON cs.customer_id=c.customer_id
      GROUP BY country
    ),
    
    country_grouped AS 
    (SELECT *,
            CASE 
            WHEN customers < 2 THEN "Other"
            ELSE country
            END AS country_grouping
       FROM country_sales),
    
    country_grouped_sales AS
    (SELECT country_grouping country, 
            SUM(customers) customers,
            SUM(sales_amount) sales_amount,
            COUNT(sales) sales,
            CASE
                WHEN country_grouping = "Other" THEN 2
                ELSE 0
            END AS sort
       FROM country_grouped
      GROUP BY country_grouping
    )
    
SELECT country,
       customers,
       "$"||sales_amount sales_amount,
       "$"||ROUND(sales_amount/customers,2) avg_sale_customer,
       "$"||ROUND(sales_amount/sales,2) avg_order
  FROM country_grouped_sales
 ORDER BY sort, sales_amount DESC;
    

 * sqlite:///chinook.db
Done.


country,customers,sales_amount,avg_sale_customer,avg_order
Canada,8,$535.59,$66.95,$535.59
Brazil,5,$427.68,$85.54,$427.68
France,5,$389.07,$77.81,$389.07
Germany,4,$334.62,$83.66,$334.62
Czech Republic,2,$273.24,$136.62,$273.24
United Kingdom,3,$245.52,$81.84,$245.52
Portugal,2,$185.13,$92.57,$185.13
India,2,$183.15,$91.58,$183.15
USA,13,$1040.49,$80.04,$1040.49
Other,15,$1094.94,$73.0,$73.0


Based on the above table there may be opportunity in Czech Republic, United Kingdom, and India. However, because the data from each of these countries is relatively low, we should be cautious about spending too much money on new marketing campaigns as the sample size isn't large enough to give us high confidence. A better approach would be to run small campaigns in these countries, collecting and analyzing data from new customers to test if these trends continue to hold with new customers.


## Analyzing Consumer Preference: What Fraction of Sales are Entire Albums?

In [85]:
%%sql

WITH 

-- Add album id information for every track found in each invoice
    invoice_info AS
    (SELECT invoice_id, il.track_id, t.album_id
       FROM invoice_line il
       LEFT JOIN track t ON t.track_id=il.track_id
    ),
    
-- Find the numbers of tracks for each album id    
    album_info AS
    (SELECT album_id, COUNT(track_id) album_track_count
       FROM track
      GROUP BY album_id
    ),

-- Add the track count for every album tied to individual tracks found in each invoice    
    invoice_album_info AS
    (SELECT *
       FROM invoice_info i
       LEFT JOIN album_info a ON a.album_id=i.album_id     
    ),
  
-- Create a table that for each invoice: 
-- 1) tallies the number of different albums the tracks in it belong to
-- 2) picks out a single sample track and shows the id and number of tracks for the album this sample track is from 
    invoice_album_track_counts AS
    (SELECT invoice_id,
            COUNT(DISTINCT album_id) invoice_album_count,
            COUNT(track_id) invoice_track_count,
            track_id sample_track_id,
            album_id sample_track_album_id,
            album_track_count sample_album_track_count
       FROM invoice_album_info
      GROUP BY invoice_id
      ),
  
--Add a column to the last table that categorizes whether each invoice was an entire album purchase or just a collection of singles
    single_v_album AS
    (SELECT *,
            CASE
                WHEN (invoice_album_count=1 AND invoice_track_count=sample_album_track_count) THEN "album"
                ELSE "single"
                END AS purchase_type
     FROM invoice_album_track_counts
    )
    
           
SELECT purchase_type,
       COUNT(invoice_id) number_of_invoices,
       ROUND(CAST(COUNT(invoice_id) AS FLOAT)/ (SELECT COUNT(invoice_id) FROM single_v_album),2)*100 invoice_percentage
FROM single_v_album
 GROUP BY purchase_type;


 * sqlite:///chinook.db
Done.


purchase_type,number_of_invoices,invoice_percentage
album,114,19.0
single,500,81.0


Approximately one-fifth of purchases are from sales of entire albums, making this purchase type a significant driver of sales. Continuing to purchase all tracks from an album, instead of only popular tracks, is a good strategy as there is demand for the whole album.